In [ ]:
## environment preparation
##============================

%reset -f

%connect_info #might be used for "jupyter-console"

In [ ]:
## Import libraries
##============================

import numpy as np
import pandas as pd
import sys, os, random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Conv1D, Conv2D, Flatten, Reshape, MaxPooling1D, MaxPooling2D, BatchNormalization, TimeDistributed
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
## Settings and Paths
##============================

path_ipynb = os.getcwd()
path_base = path_ipynb + "/../../data/"

## the filename with data is defined and presented at the end of
## "03_Data_Balance_Upsample_Save_to_Single_File.ipynb"
path_add_folder_with_data = "processed_data/"
filename2load = "demo_run_xdf_20211218_cle_bal_noCnoA1_normal.csv"

path_add_folder2savemodel = "tf_models/"
model_filename = 'model_demo_run'

In [ ]:
## Load Data
##============================

Xdf = pd.read_csv(path_base + path_add_folder_with_data + filename2load)

In [ ]:
Xdf.head()

In [ ]:
## Split to Data and Labels
##============================

X_init = Xdf.iloc[:,1:7]
Y_init = Xdf.label

In [ ]:
## Lets see the classes
##============================

classes_values = Xdf.label.unique().tolist()
classes_values.sort()
classes = len(classes_values)
classes_values

In [ ]:
## Dictionary to keep labels
##============================

dict_4labels = dict.fromkeys(classes_values, "default")
d_counter = 0
for d_key in dict_4labels:
    d_counter += 1
    dict_4labels[d_key]  = d_counter

dict_4labels

In [ ]:
## Data converting and reshaping for training
##============================


# Set random seeds for repeatable results
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)




Y = tf.keras.utils.to_categorical(Y_init.replace(dict_4labels)-1, classes)
X = X_init.to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

input_length = X_train[0].shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

callbacks = []


In [ ]:
## Create a model and compile it
##============================


# model architecture
model = Sequential()
model.add(Dense(36, 
                activation='relu', 
                activity_regularizer=tf.keras.regularizers.l1(0.00001)))
model.add(Dropout(0.2))
model.add(Dense(24, 
                activation='relu',
                activity_regularizer=tf.keras.regularizers.l1(0.00001)))
model.add(Dropout(0.2))
model.add(Dense(classes, 
                activation='softmax', 
                name='y_pred'))

# this controls the learning rate
opt = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999)

# this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=False)
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=False)

# train the neural network
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [ ]:
## Fit! 
## The training process is here. 
##============================

model.fit(train_dataset, epochs=15, validation_data=validation_dataset, verbose=2, callbacks=callbacks)

In [ ]:
## Lets see the model summary
##============================

model.summary()

In [ ]:
## Evaluate the model
##============================

loss, acc = model.evaluate(X_test, Y_test, verbose=2)
print("Model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
## Save the model to disk
##============================

# model.save('saved_model')
# !mkdir -p saved_model

model.save(path_base + path_add_folder2savemodel + model_filename )

In [ ]:
## Cross check that the model is saved correctly
##============================


## Load into a new model
new_model = tf.keras.models.load_model(path_base + path_add_folder2savemodel + model_filename )

# Check its architecture
new_model.summary()

In [ ]:
## Re-evaluate the model
##============================

loss, acc = new_model.evaluate(X_test, Y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(X_test).shape)

In [ ]:
##################################################
##### Some tests to cross-check the model
##################################################

In [ ]:
##### random sample from TRAIN set
randt_index_X = np.random.randint(low=0, high=X_train.shape[0], size=1, dtype=int)
print(f"Values_Train: {X_train[randt_index_X,:]}")
print(f"Label_Train:  {Y_train[randt_index_X]}")
print(f"Label_Text:   {pd.DataFrame(classes_values).loc[[bool(cc) for cc in Y_train[randt_index_X].tolist()[0]],:].values}")

In [ ]:
##### random sample from TEST set
randt_index_X = np.random.randint(low=0, high=X_test.shape[0], size=1, dtype=int)
print(f"Values_Test: {X_test[randt_index_X,:]}")
print(f"Label_Test:  {Y_test[randt_index_X]}")
print(f"Label_Text:   {pd.DataFrame(classes_values).loc[[bool(cc) for cc in Y_test[randt_index_X].tolist()[0]],:].values}")

In [ ]:
##### random sample from TEST set
randt_index_X = np.random.randint(low=0, high=X_test.shape[0], size=1, dtype=int)
print(f"Values_Test: {X_test[randt_index_X,:]}")
print(f"Label_Test:  {Y_test[randt_index_X]}")
print(f"Label_Text:   {pd.DataFrame(classes_values).loc[[bool(cc) for cc in Y_test[randt_index_X].tolist()[0]],:].values}")

In [ ]:
randt_index_X = np.random.randint(low=0, high=X_init.shape[0], size=1, dtype=int)

print(f"X sample: {X_init.loc[randt_index_X, :]} \nY label: {Y_init.loc[randt_index_X]}")